### Import Library & Connect to Google

In [1]:
# pip install pytrends
import pandas as pd
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

### Keyword List

In [2]:
directory = "/Users/rfdanti/Documents/Online Course/GoogleTrends/"
filename = "list_keywords_food.csv"
kw_file = pd.read_csv(directory+filename)
kw_list = kw_file["Keywords"].values.tolist()

In [3]:
kw_file.head()

,Keywords,Type,Origin
0,Burger,Mains,American
1,Grilled Cheese,Mains,American
2,Hot dog,Mains,American
3,Mac and Cheese,Mains,American
4,Cheesecake,Desserts/Drinks,American


### Building Payload & Collecting Data of Keywords

In [4]:
def gtrends_overtime(full_list, key_ref, save_name="", directory="", category=0, time='all', loc=''):
    #iterate every 4 item in list plus a keyword reference as the relative comparison
    i = 0
    while i < len(kw_list):
        l = kw_list[i:(i+4)]
        l.append(key_ref)
        pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
        df_time = pytrends.interest_over_time()
        df_time.reset_index(inplace=True)
        df_time_name = "gtrends_overtime"+str(save_name)+str((i+4)//4)+".csv"
        df_time.to_csv(directory+df_time_name, index = False)
        i += 4

In [ ]:
gtrends_overtime(kw_list, 'Noodles', "_worldwide_", directory,
                 category=71, time='all', loc='')

In [31]:
gtrends_overtime(kw_list, 'Noodles', "_US_", directory,
                 category=71, time='all', loc='US')

In [32]:
gtrends_overtime(kw_list, 'Noodles', "_ID_", directory,
                 category=71, time='all', loc='ID')

In [6]:
def gtrends_region(kw_list, key_ref, save_name="", directory="",
                   category=0, time='all', loc='', res='COUNTRY'):
    #iterate every 4 item in list plus a keyword reference as the relative comparison
    i = 0
    while i < len(kw_list):
        l = kw_list[i:(i+4)]
        l.append(key_ref)
        pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
        # resolution can be set to 'REGION' when loc is set to specific country, e.g. 'US'
        df_region = pytrends.interest_by_region(resolution=res, inc_low_vol=True, inc_geo_code=False)
        df_region.reset_index(inplace=True)
        df_region_name = "gtrends_region"+str(save_name)+str((i+4)//4)+".csv"
        df_region.to_csv(df_region_name, index = False)
        i += 4

In [7]:
gtrends_region(kw_list, 'Noodles', "_worldwide_", directory,
               category=71, time='all', loc='', res='COUNTRY')

In [27]:
gtrends_region(kw_list, 'Noodles', "_US_", directory,
               category=71, time='all', loc='US', res='REGION')

In [29]:
gtrends_region(kw_list, 'Noodles', "_ID_", directory,
               category=71, time='all', loc='ID', res='REGION')

### Integration, Normalization & Cleaning Data

In [14]:
def combine_wbase(directory, base_name, n_file, filename):
    df1 = pd.read_csv(directory+base_name+str(1)+".csv")
    for i in range(n_file-1):
        df2 = pd.read_csv(directory+base_name+str(i+2)+".csv")
        df1 = pd.concat([df1, df2], axis=1, sort=False)
    df_name = filename
    # Saving the merged file or you can simply return the dataframe
    df1.to_csv(df_name, index = False)
    #return df

In [15]:
combined = combine_wbase("/Users/rfdanti/Documents/Online Course/GoogleTrends/",
              "gtrends_overtime_worldwide_", 25, "gtrends_overtime_worlwide_merged.csv")
combined = pd.read_csv("gtrends_overtime_worlwide_merged.csv")
combined

,date,Burger,Grilled Cheese,Hot dog,Mac and Cheese,Noodles,isPartial,date.1,Cheesecake,Chow Mein,...,Pad Thai,Noodles.23,isPartial.23,date.24,Tom Yum,Khao Pad,Thai Curry,Mango Sticky Rice,Noodles.24,isPartial.24
0,2004-01-01,15,0,3,0,5,False,2004-01-01,26,2,...,4,17,False,2004-01-01,2,0,6,0,17,False
1,2004-02-01,14,0,3,0,5,False,2004-02-01,29,1,...,4,16,False,2004-02-01,1,0,6,1,16,False
2,2004-03-01,16,0,3,0,5,False,2004-03-01,25,2,...,4,16,False,2004-03-01,0,0,5,0,16,False
3,2004-04-01,18,0,3,0,4,False,2004-04-01,30,1,...,4,13,False,2004-04-01,0,0,4,1,13,False
4,2004-05-01,17,0,3,0,5,False,2004-05-01,27,1,...,4,15,False,2004-05-01,1,0,3,0,15,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2020-03-01,73,2,5,10,24,False,2020-03-01,58,4,...,13,78,False,2020-03-01,3,0,12,1,78,False
195,2020-04-01,80,3,7,13,30,False,2020-04-01,99,6,...,19,100,False,2020-04-01,4,0,17,2,100,False
196,2020-05-01,97,3,7,11,28,False,2020-05-01,100,6,...,18,94,False,2020-05-01,4,0,16,2,94,False
197,2020-06-01,90,2,7,10,26,False,2020-06-01,80,6,...,16,85,False,2020-06-01,3,0,14,1,85,False


In [50]:
combined_US = combine_wbase("/Users/rfdanti/Documents/Online Course/GoogleTrends/",
              "gtrends_overtime_US_", 25, "gtrends_overtime_US_merged.csv")
combined_US = pd.read_csv("gtrends_overtime_US_merged.csv")

combined_ID = combine_wbase("/Users/rfdanti/Documents/Online Course/GoogleTrends/",
              "gtrends_overtime_ID_", 25, "gtrends_overtime_ID_merged.csv")
combined_ID = pd.read_csv("gtrends_overtime_ID_merged.csv")

In [16]:
combined_geo = combine_wbase("/Users/rfdanti/Documents/Online Course/GoogleTrends/",
              "gtrends_region_worldwide_", 25, "gtrends_region_worlwide_merged.csv")
combined_geo = pd.read_csv("gtrends_region_worlwide_merged.csv")
combined_geo

,geoName,Burger,Grilled Cheese,Hot dog,Mac and Cheese,Noodles,geoName.1,Cheesecake,Chow Mein,Dimsum,...,Paella,Leche Frita,Pad Thai,Noodles.23,geoName.24,Tom Yum,Khao Pad,Thai Curry,Mango Sticky Rice,Noodles.24
0,Afghanistan,58,0,12,0,30,Afghanistan,41,0,0,...,0,0,0,100,Afghanistan,0,0,0,0,100
1,Albania,56,0,10,6,28,Albania,79,0,0,...,28,0,0,65,Albania,0,0,0,0,100
2,Algeria,70,0,18,2,10,Algeria,89,0,0,...,87,0,0,11,Algeria,0,0,0,0,100
3,American Samoa,0,0,0,0,0,American Samoa,0,0,0,...,0,0,0,0,American Samoa,0,0,0,0,0
4,Andorra,91,0,0,0,9,Andorra,76,0,0,...,32,4,0,5,Andorra,0,0,0,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Western Sahara,0,0,0,0,0,Western Sahara,0,0,0,...,0,0,0,0,Western Sahara,0,0,0,0,0
246,Yemen,70,0,0,0,30,Yemen,57,0,0,...,0,0,0,100,Yemen,0,0,0,0,100
247,Zambia,43,0,9,14,34,Zambia,51,0,0,...,0,0,0,100,Zambia,0,0,0,0,100
248,Zimbabwe,54,0,6,12,28,Zimbabwe,55,0,0,...,0,0,0,100,Zimbabwe,0,0,0,0,100


In [90]:
combined_geo_US = combine_wbase("/Users/rfdanti/Documents/Online Course/GoogleTrends/Data/",
              "gtrends_region_US_", 25, "gtrends_region_US_merged.csv")
combined_geo_US = pd.read_csv("gtrends_region_US_merged.csv")
combined_geo_US.head()

,geoName,Burger,Grilled Cheese,Hot dog,Mac and Cheese,Noodles,geoName.1,Cheesecake,Chow Mein,Dimsum,...,Paella,Leche Frita,Pad Thai,Noodles.23,geoName.24,Tom Yum,Khao Pad,Thai Curry,Mango Sticky Rice,Noodles.24
0,Alabama,62,2,6,12,18,Alabama,61,3,0,...,5,0,6,88,Alabama,1,0,4,0,95
1,Alaska,52,3,6,14,25,Alaska,49,5,0,...,4,0,39,57,Alaska,0,0,15,0,85
2,Arizona,64,2,6,10,18,Arizona,61,4,0,...,6,0,10,81,Arizona,4,0,8,0,88
3,Arkansas,62,3,5,10,20,Arkansas,55,3,0,...,4,0,11,83,Arkansas,1,0,7,0,92
4,California,64,3,5,9,19,California,60,5,0,...,9,0,12,76,California,3,0,11,1,85


In [91]:
combined_geo_ID = combine_wbase("/Users/rfdanti/Documents/Online Course/GoogleTrends/Data/",
              "gtrends_region_ID_", 25, "gtrends_region_ID_merged.csv")
combined_geo_ID = pd.read_csv("gtrends_region_ID_merged.csv")
combined_geo_ID.head()

,geoName,Burger,Grilled Cheese,Hot dog,Mac and Cheese,Noodles,geoName.1,Cheesecake,Chow Mein,Dimsum,...,Paella,Leche Frita,Pad Thai,Noodles.23,geoName.24,Tom Yum,Khao Pad,Thai Curry,Mango Sticky Rice,Noodles.24
0,Aceh,100,0,0,0,0,Aceh,4,0,96,...,0,0,0,0,Aceh,0,0,0,0,0
1,Bali,78,0,5,2,15,Bali,26,0,45,...,11,0,21,61,Bali,22,0,0,10,68
2,Bangka Belitung Islands,100,0,0,0,0,Bangka Belitung Islands,31,0,69,...,0,0,0,0,Bangka Belitung Islands,0,0,0,0,0
3,Banten,88,0,2,4,6,Banten,21,0,73,...,11,0,16,70,Banten,29,0,0,25,46
4,Bengkulu,100,0,0,0,0,Bengkulu,14,0,86,...,0,0,0,0,Bengkulu,0,0,0,0,0


In [17]:
# Tidy up IsPartial column for data overtime
def partial(df, n_file):
    for i in range(n_file-1):
        df = df.drop(columns="isPartial."+str(i+1)+"")
    if df.isPartial.tail(1).bool() == True:
        df = df.drop(df.isPartial.tail(1).index, axis=0)
    df = df.drop(columns="isPartial")
    return df

In [18]:
combined = partial(combined,25)
combined

,date,Burger,Grilled Cheese,Hot dog,Mac and Cheese,Noodles,date.1,Cheesecake,Chow Mein,Dimsum,...,Paella,Leche Frita,Pad Thai,Noodles.23,date.24,Tom Yum,Khao Pad,Thai Curry,Mango Sticky Rice,Noodles.24
0,2004-01-01,15,0,3,0,5,2004-01-01,26,2,0,...,12,1,4,17,2004-01-01,2,0,6,0,17
1,2004-02-01,14,0,3,0,5,2004-02-01,29,1,0,...,13,1,4,16,2004-02-01,1,0,6,1,16
2,2004-03-01,16,0,3,0,5,2004-03-01,25,2,1,...,12,1,4,16,2004-03-01,0,0,5,0,16
3,2004-04-01,18,0,3,0,4,2004-04-01,30,1,1,...,10,1,4,13,2004-04-01,0,0,4,1,13
4,2004-05-01,17,0,3,0,5,2004-05-01,27,1,0,...,13,1,4,15,2004-05-01,1,0,3,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2020-02-01,84,2,6,9,24,2020-02-01,59,5,2,...,19,1,14,78,2020-02-01,3,0,12,1,78
194,2020-03-01,73,2,5,10,24,2020-03-01,58,4,2,...,17,1,13,78,2020-03-01,3,0,12,1,78
195,2020-04-01,80,3,7,13,30,2020-04-01,99,6,5,...,28,5,19,100,2020-04-01,4,0,17,2,100
196,2020-05-01,97,3,7,11,28,2020-05-01,100,6,5,...,29,1,18,94,2020-05-01,4,0,16,2,94


In [51]:
combined_US = partial(combined_US,25)
combined_ID = partial(combined_ID,25)

normalisation formula => 
new_value[column][n] = old_value[column][n] * (df[key_ref][n]/df[key_ref+"."+str(i)+""][n])

In [112]:
def normalise(df, n_file, key_ref, col='date'):
    li = []
    # Checking the relative popularity between comparisons
    for i in range(n_file-1):    
        df = df.drop(columns=col+"."+str(i+1)+"")
        # Appending the list if relative popularity of the keyword reference is different
        if df[key_ref+"."+str(i+1)+""][0] == df[key_ref][0]:
            pass
        else:
            li.append(i+1)
    
    # Normalizing relative popularity when the relative popularity of the keyword reference is different         
    for l in li:
        k = df.columns.get_loc(key_ref+"."+str(l)+"")
        for n in range(len(df.index)):
            # Computing relative popularity by normalizing according to the reference
            if df.iloc[n,(k)] > 0:
                for m in range(5):
                    df.iloc[n,(k-4+m)] = (df.iloc[n,(k-4+m)] * (df[key_ref][n]/df.iloc[n,(k)]))
            else:
                for m in range(5):
                    df.iloc[n,(k-4+m)] = (df.iloc[n,(k-4+m)] * (df[key_ref][n]/0.01))
    return df

In [113]:
normalised = normalise(combined, n_file=25, key_ref="Noodles", col='date')
normalised

,date,Burger,Grilled Cheese,Hot dog,Mac and Cheese,Noodles,Cheesecake,Chow Mein,Dimsum,Vermicelli roll,...,Jamon,Paella,Leche Frita,Pad Thai,Noodles.23,Tom Yum,Khao Pad,Thai Curry,Mango Sticky Rice,Noodles.24
0,2004-01-01,15,0,3,0,5,13.000000,1.000000,0.000000,0.0,...,1.764706,3.529412,0.294118,1.176471,5.0,0.588235,0.0,1.764706,0.000000,5.0
1,2004-02-01,14,0,3,0,5,16.111111,0.555556,0.000000,0.0,...,1.562500,4.062500,0.312500,1.250000,5.0,0.312500,0.0,1.875000,0.312500,5.0
2,2004-03-01,16,0,3,0,5,13.888889,1.111111,0.555556,0.0,...,1.250000,3.750000,0.312500,1.250000,5.0,0.000000,0.0,1.562500,0.000000,5.0
3,2004-04-01,18,0,3,0,4,15.000000,0.500000,0.500000,0.0,...,0.923077,3.076923,0.307692,1.230769,4.0,0.000000,0.0,1.230769,0.307692,4.0
4,2004-05-01,17,0,3,0,5,15.000000,0.555556,0.000000,0.0,...,1.333333,4.333333,0.333333,1.333333,5.0,0.333333,0.0,1.000000,0.000000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2020-02-01,84,2,6,9,24,31.466667,2.666667,1.066667,0.0,...,4.615385,5.846154,0.307692,4.307692,24.0,0.923077,0.0,3.692308,0.307692,24.0
194,2020-03-01,73,2,5,10,24,30.933333,2.133333,1.066667,0.0,...,4.923077,5.230769,0.307692,4.000000,24.0,0.923077,0.0,3.692308,0.307692,24.0
195,2020-04-01,80,3,7,13,30,52.105263,3.157895,2.631579,0.0,...,6.900000,8.400000,1.500000,5.700000,30.0,1.200000,0.0,5.100000,0.600000,30.0
196,2020-05-01,97,3,7,11,28,52.830189,3.169811,2.641509,0.0,...,5.659574,8.638298,0.297872,5.361702,28.0,1.191489,0.0,4.765957,0.595745,28.0


In [114]:
normalised_US = normalise(combined_US, n_file=25, key_ref="Noodles", col='date')
normalised_ID = normalise(combined_ID, n_file=25, key_ref="Noodles", col='date')

In [116]:
normalised_geo = normalise(combined_geo, n_file=25, key_ref="Noodles", col='geoName')
normalised_geo

,geoName,Burger,Grilled Cheese,Hot dog,Mac and Cheese,Noodles,Cheesecake,Chow Mein,Dimsum,Vermicelli roll,...,Jamon,Paella,Leche Frita,Pad Thai,Noodles.23,Tom Yum,Khao Pad,Thai Curry,Mango Sticky Rice,Noodles.24
0,Afghanistan,58,0,12,0,30,20.847458,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,30.0,0.0,0.0,0.0,0.0,30.0
1,Albania,56,0,10,6,28,105.333333,0.0,0.0,0.0,...,3.015385,12.061538,0.0,0.0,28.0,0.0,0.0,0.0,0.0,28.0
2,Algeria,70,0,18,2,10,80.909091,0.0,0.0,0.0,...,1.818182,79.090909,0.0,0.0,10.0,0.0,0.0,0.0,0.0,10.0
3,American Samoa,0,0,0,0,0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Andorra,91,0,0,0,9,28.500000,0.0,0.0,0.0,...,106.200000,57.600000,7.2,0.0,9.0,0.0,0.0,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Western Sahara,0,0,0,0,0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,Yemen,70,0,0,0,30,39.767442,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,30.0,0.0,0.0,0.0,0.0,30.0
247,Zambia,43,0,9,14,34,35.387755,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,34.0,0.0,0.0,0.0,0.0,34.0
248,Zimbabwe,54,0,6,12,28,34.222222,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,28.0,0.0,0.0,0.0,0.0,28.0


In [115]:
normalised_geo_US = normalise(combined_geo_US, n_file=25, key_ref="Noodles", col='geoName')
normalised_geo_ID = normalise(combined_geo_ID, n_file=25, key_ref="Noodles", col='geoName')

In [117]:
def tidy(df, n_file, key_ref, kw_file, col='date'):
    for i in range(n_file-1):    
        df = df.drop(columns=key_ref+"."+str(i+1)+"")
    df=pd.melt(df,id_vars=[col],var_name='Keywords', value_name='RelativePopularity')
    df = df.merge(kw_file, on="Keywords")
    return df

In [118]:
overtime = tidy(normalised, 25, "Noodles", kw_file, col='date')
overtime

,date,Keywords,RelativePopularity,Type,Origin
0,2004-01-01,Burger,15.000000,Mains,American
1,2004-02-01,Burger,14.000000,Mains,American
2,2004-03-01,Burger,16.000000,Mains,American
3,2004-04-01,Burger,18.000000,Mains,American
4,2004-05-01,Burger,17.000000,Mains,American
...,...,...,...,...,...
19795,2020-02-01,Mango Sticky Rice,0.307692,Desserts/Drinks,Thai
19796,2020-03-01,Mango Sticky Rice,0.307692,Desserts/Drinks,Thai
19797,2020-04-01,Mango Sticky Rice,0.600000,Desserts/Drinks,Thai
19798,2020-05-01,Mango Sticky Rice,0.595745,Desserts/Drinks,Thai


In [119]:
overtime_US = tidy(normalised_US, 25, "Noodles", kw_file, col='date')
overtime_ID = tidy(normalised_ID, 25, "Noodles", kw_file, col='date')

In [120]:
tidied_geo = tidy(normalised_geo, 25, "Noodles", kw_file, col='geoName')
tidied_geo

,geoName,Keywords,RelativePopularity,Type,Origin
0,Afghanistan,Burger,58.0,Mains,American
1,Albania,Burger,56.0,Mains,American
2,Algeria,Burger,70.0,Mains,American
3,American Samoa,Burger,0.0,Mains,American
4,Andorra,Burger,91.0,Mains,American
...,...,...,...,...,...
24995,Western Sahara,Mango Sticky Rice,0.0,Desserts/Drinks,Thai
24996,Yemen,Mango Sticky Rice,0.0,Desserts/Drinks,Thai
24997,Zambia,Mango Sticky Rice,0.0,Desserts/Drinks,Thai
24998,Zimbabwe,Mango Sticky Rice,0.0,Desserts/Drinks,Thai


In [121]:
geo_US = tidy(normalised_geo_US, 25, "Noodles", kw_file, col='geoName')
geo_ID = tidy(normalised_geo_ID, 25, "Noodles", kw_file, col='geoName')

In [122]:
overtime["Loc"] = "Worldwide"
overtime_US["Loc"] = "US"
overtime_ID["Loc"] = "ID"

overtime_all = pd.concat([overtime, overtime_US], axis=0, sort=False)
overtime_all = pd.concat([overtime_all, overtime_ID], axis=0, sort=False)

overtime_all.to_csv("gtrends_food_overtime_all.csv", index = False)
overtime_all

,date,Keywords,RelativePopularity,Type,Origin,Loc
0,2004-01-01,Burger,15.000000,Mains,American,Worldwide
1,2004-02-01,Burger,14.000000,Mains,American,Worldwide
2,2004-03-01,Burger,16.000000,Mains,American,Worldwide
3,2004-04-01,Burger,18.000000,Mains,American,Worldwide
4,2004-05-01,Burger,17.000000,Mains,American,Worldwide
...,...,...,...,...,...,...
19795,2020-02-01,Mango Sticky Rice,0.875000,Desserts/Drinks,Thai,ID
19796,2020-03-01,Mango Sticky Rice,0.857143,Desserts/Drinks,Thai,ID
19797,2020-04-01,Mango Sticky Rice,0.875000,Desserts/Drinks,Thai,ID
19798,2020-05-01,Mango Sticky Rice,0.888889,Desserts/Drinks,Thai,ID


In [123]:
geo = tidied_geo.fillna(0)
geoSum = pd.DataFrame(geo.groupby(['geoName']).sum()).reset_index()
geoSum.columns = ['geoName', 'geoSum']
geoSum

,geoName,geoSum
0,Afghanistan,430.135493
1,Albania,2175.964817
2,Algeria,2475.080297
3,American Samoa,0.000000
4,Andorra,1152.239261
...,...,...
245,Western Sahara,0.000000
246,Yemen,401.735841
247,Zambia,657.143670
248,Zimbabwe,863.195132


In [124]:
geo = pd.merge(geo, geoSum, on=['geoName'])
geo['PercentageBreakdown'] = geo['RelativePopularity']/geo['geoSum']
geo.to_csv("gtrends_food_geo.csv", index = False)
geo

,geoName,Keywords,RelativePopularity,Type,Origin,geoSum,PercentageBreakdown
0,Afghanistan,Burger,58.000000,Mains,American,430.135493,0.134841
1,Afghanistan,Grilled Cheese,0.000000,Mains,American,430.135493,0.000000
2,Afghanistan,Hot dog,12.000000,Mains,American,430.135493,0.027898
3,Afghanistan,Mac and Cheese,0.000000,Mains,American,430.135493,0.000000
4,Afghanistan,Cheesecake,20.847458,Desserts/Drinks,American,430.135493,0.048467
...,...,...,...,...,...,...,...
24995,Åland Islands,Pad Thai,0.000000,Soup/Noodles,Thai,100.000000,0.000000
24996,Åland Islands,Tom Yum,0.000000,Soup/Noodles,Thai,100.000000,0.000000
24997,Åland Islands,Khao Pad,0.000000,Mains,Thai,100.000000,0.000000
24998,Åland Islands,Thai Curry,0.000000,Mains,Thai,100.000000,0.000000


In [126]:
geo_US = geo_US.fillna(0)
geo_US.to_csv("gtrends_food_geo_US.csv", index = False)

geo_ID = geo_ID.fillna(0)
geo_ID.to_csv("gtrends_food_geo_ID.csv", index = False)

### Check out the DataViz in Tableau public: 
https://public.tableau.com/profile/ristanti.ramadanti#!/vizhome/100FoodAroundTheWorld/Dashboard1